In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [ ]:
#model variables
test_train_split = 0.75
batch_size = 64
epoch_number = 100
validation_split = 0.15
learning_rate = 2e-6
dropout_rate = 0.45

#architecture variables
CNL1_filters = 32
CNL1_kernal_size = 5
MPL1_pool_size= (2,2)
MPL1_strides = 2
CNL2_filters = 64
CNL2_kernal_size = 5
MPL2_pool_size = (2,2)
MPL2_strides = 2

In [3]:
galaxyzoo = pd.read_csv("zoo2MainSpecz.csv/zoo2MainSpecz.csv", usecols=[8], nrows=10000)
Class = galaxyzoo["gz2class"].values
dictionary = {'A':int(2),'E':int(1),'S':int(0)}
#resave using my dictionary
target = np.empty((len(Class)))
for i in range(len(Class)):
    target[i] = int(dictionary[Class[i][0]])

In [ ]:
#https://keras.io/preprocessing/image/

#datagen flow from directory wont work, need to create a different iterator. Have to combine with the targets somehow. 



In [ ]:
def create_model(dropout_rate=dropout_rate, learning_rate=learning_rate):
    
    model = keras.Sequential([])
    model.add(keras.layers.Conv2D(input_shape=(64,64,5),filters=CNL1_filters,kernel_size=CNL1_kernal_size,padding="same",activation=tf.nn.relu))
    model.add(keras.layers.MaxPool2D(pool_size=MPL1_pool_size, strides=MPL1_strides))
    model.add(keras.layers.Conv2D(filters=CNL2_filters,kernel_size=CNL2_kernal_size,padding="same",activation=tf.nn.relu))
    model.add(keras.layers.MaxPool2D(pool_size=MPL2_pool_size,strides=MPL2_strides))
    model.add(keras.layers.Reshape([7*7*64]))
    model.add(keras.layers.Dense(units=1024,activation=tf.nn.relu))
    model.add(keras.layers.Dropout(rate=dropout_rate))
    model.add(keras.layers.Dense(units=3,activation=tf.nn.softmax))
    adam = keras.optimizers.Adam(lr = learning_rate)
    model.compile(optimizer=adam, loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return(model)

In [ ]:
#datagenerator will rotate flip and zoom my data for augmentations, b/c of cosmo principle I must be invarient to these.
datagen = keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=180,
    zoom_range=0.3,
    fill_mode='nearest',
    horizontal_flip=True,
    vertical_flip=True,
    data_format='channels_last',
    validation_split=validation_split)

datagen.fit(train_images)

In [ ]:
model = create_model(dropout_rate=dropout_rate, learning_rate = learning_rate)

history = model.fit_generator(datagen.flow(train_images, train_target, batch_size=batch_size),
                    steps_per_epoch=len(train_target) / batch_size,
                    epochs=epoch_number,
                    validation_data=datagen.flow(train_images, train_target))

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_target)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)